In [1]:
import matplotlib.pyplot as plt
import numpy as np
import jieba.analyse
import jieba
import codecs
import pandas as pd
import re

# Load lyric data base

In [38]:
df = pd.read_csv ('./data/lyric_df.csv')
df

,Singer,Name,Lyric,Category,seg_text,remove_synonym,top_10
0,A-Lin,One Life,白雲撞進藍天 激起昨天畫面我想起你的臉好久不見 在遠方的My Friend不論晴天雨天 也會...,友情,"白雲撞, 進藍天, 激起, 昨天, 畫面, 想起, 臉, 好久, 不見, 遠方, MyFri...",白雲撞 進藍天 激起 昨天 畫面 想起 臉 好久 不見 遠處 MyFriend 晴天 雨天 ...,"['Life', '我們', '一天', '期待', '哪邊', '邊要', '出現', '..."
1,A-Lin,最好的朋友在身邊,一起笑一起哭一起鬧久別的回憶又浮現一點點一天天一年年最好的朋友在身邊一起走一起跑一起跳相聚到...,友情,"一起, 笑, 一起, 哭, 一起, 鬧久別, 回憶, 浮現, 一點點, 一天天, 一年, 年...",一起 笑 一起 哭 一起 鬧久別 回憶 浮現 一些 一天天 一年 年 最好 朋友 身邊 一起...,"['一起', '一天天', '永在', '鬧久別', '回憶', '浮現', '一點點', ..."
2,A-Lin,愛的可能,你出現我身邊 像個奇蹟發生沒想到會是你 讓我如此失魂我心中的感覺是這樣陌生快樂的牽掛在相聚的...,友情,"出現, 身邊, 蹟, 發生, 想到, 失魂, 心中, 感覺, 陌生, 快樂, 牽掛, 相聚,...",出現 身邊 蹟 發生 想 失魂 心中 感覺 陌生 快樂 牽掛 相見 每一分 曾以 見過 所有...,"['我會', '不會', '記得', '孤單', '時候', '因為', '還有', '我的..."
3,A-Lin,太太太耐斯,孩子還沒睡 拜託別鬧手遊聲太吵 狗狗在叫碗盤在洗碗槽 主婦的煩惱惱公不瞭 他先洗澡奶瓶把香水...,反抗反駁反諷,"孩子, 還沒, 睡, 拜, 託, 別鬧, 手遊聲, 太吵, 狗狗, 碗, 盤, 洗碗, 槽,...",孩子 還沒 睡 拜 託 別鬧 手遊聲 太吵 狗狗 碗 盤 洗碗 槽 主婦 煩惱 惱公 瞭 先...,"['太太', '耐斯', '什麼', '超過', '太過', '發脾氣', '心軟', '忘..."
4,A-Lin,抱歉 我不抱歉,恨 什麼叫恨 這個單字 好陌生我只記得 一路走來 愛的人就算他們 多可恨等 雖然溫柔 還打不...,反抗反駁反諷,"恨, 恨, 單字, 好, 陌生, 只, 記得, 一路, 走來, 愛的, 人, 可恨, 溫柔,...",恨 恨 單字 好 陌生 只 記得 一路 走過來 愛的 人 可恨 溫柔 還打 不開 一些 門雖...,"['抱歉', '他們', '每枝', '什麼', '我們', '每個', '每張', '太真..."
...,...,...,...,...,...,...,...
2201,陳奕迅,黑暗中漫舞,為何未能學會起舞便已抱緊你誰料到資質不配合你左腳舉起了便要別離為何未能讓我衰老便要放開你陪你...,離別,"未能, 學會, 起舞, 已, 抱, 緊, 料到, 資質, 配合, 左腳, 舉起, 別離, 未...",未能 學 起舞 已經 抱 緊 料到 資質 配合 左腳 舉起 別離 未能 衰老 放開 陪伴 跳...,"['為何', '未能', '左腳', '美不美', '捱到', '學會', '資質', '舉..."
2202,陳奕迅,傷信,重讀著你的告別信 抑壓了暗湧雖不信寫的話 竟可以這麼重但再哭亦無用 徐徐又當這信是你 緊貼我...,離別,"重讀, 告別, 信, 抑壓, 暗湧, 雖不信, 寫, 話, 重, 再, 哭, 亦, 無用, ...",重讀 告別 信 抑壓 暗湧 雖不信 寫 話 重 再 哭 亦 無用 徐徐 又當 這信 緊貼 抱...,"['抑壓', '怎麼', '重讀', '告別', '暗湧', '雖不信', '這麼', '無..."
2203,陳奕迅,歲月如歌,愛上了 看見你 如何不懂謙卑去講心中理想 不會俗氣猶如看得見晨曦 才能歡天喜地抱著你 我每次...,離別,"愛上, 懂謙卑, 去講, 心中, 理想, 俗氣, 猶如, 晨曦, 歡天, 喜地, 抱, 每次...",愛上 懂謙卑 去講 心中 夢想 俗氣 猶如 晨曦 歡天 喜地 抱 每次 回來 驚喜 也許 一...,"['也許', '愛上', '看見', '懂謙卑', '去講', '不會', '俗氣', '猶..."
2204,陳奕迅,遇見了你(國),遇見了你 讓生命美好我看見繁華的春天愛情如陽光閃耀我給你我的愛離開了我 你默默低頭我看見往事...,離別,"遇見, 生命, 美好, 看見, 繁華, 春天, 愛情, 如陽光, 閃耀, 我給, 愛離開, ...",遇到 生命 美好 看見 繁華 春天 愛情 如陽光 閃耀 我給 愛離開 靜靜 低頭 往事 如雲...,"['離開', '我給', '帶著', '愛帶', '飛到', '邊到', '一個', '傷終..."


# Import testing lyric and preprocess

In [22]:
def remove_punctuation(line):
    stopwords = [line.strip() for line in open('./data/stopwords.txt', 'r', encoding='utf-8').readlines()]
    line = str(line)
    if line.strip() == '':
        return ''
    re_han = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = re_han.sub('', line)
    cut = [w for w in list(jieba.cut(line)) if w not in stopwords]
    res = ", ".join(cut)
    return res

In [3]:
# 載入同義字
word_net = []
with open("data/synonyms.txt", "r", encoding = "utf-8") as f1:
    for line in f1:
        word_net.append(line)

word_net = sorted(set(word_net))
synonyms_dic = {}

for word in word_net:
    word_s = word.split()
    synonyms_dic[word_s[0]] = word_s[1]

In [23]:
def synonyms(line):
    line_words = line.split(", ")
    line_lyrics = ""
    for line_word in line_words:
        if line_word in synonyms_dic:
            line_lyrics = line_lyrics + synonyms_dic[line_word] + ' '
        else:
            line_lyrics = line_lyrics + line_word + ' '
    return line_lyrics

In [24]:
def getTopTen(line):
    words = jieba.analyse.extract_tags(line, 10)
    return words

In [25]:
test_lyric = "過完整個夏天　憂傷並沒有好一些 開車行駛在公路無際無邊 有離開自己的感覺 唱不完一首歌　疲倦還剩下黑眼圈 感情的世界傷害在所難免 黃昏再美終要黑夜 依然記得從你口中說出再見堅決如鐵 昏暗中有種烈日灼身的錯覺 黃昏的地平線　劃出一句離別 愛情進入永夜 依然記得從你眼中滑落的淚傷心欲絕 混亂中有種熱淚燒傷的錯覺 黃昏的地平線　割斷幸福喜悅 相愛已經幻滅"
test_lyric_seg = remove_punctuation(test_lyric)
doc = synonyms(test_lyric_seg)
doc_top10 = getTopTen(doc)

# Load LSI and LDA models

## 載入語料庫

In [7]:
import os
from gensim import corpora, models, similarities

In [8]:
# 載入語料庫
if (os.path.exists("data/lyrics.dict")):
    dictionary = corpora.Dictionary.load("data/lyrics.dict") # 建立Dictionary
    print(dictionary)
    corpus = corpora.MmCorpus("data/lyrics.mm") # 將數據流的語料變為內容流的語料
    print("Used files generated from first tutorial")
    print(corpus)
else:
    print("Please run first tutorial to generate data set")

Dictionary(38350 unique tokens: ['MyFriend', 'OneLife', '一些', '一天', '一起']...)
Used files generated from first tutorial
MmCorpus(2206 documents, 38350 features, 131142 non-zero entries)


## 建立TF-IDF模型

In [9]:
tfidf = models.TfidfModel(corpus)
# 轉為向量表示
corpus_tfidf = tfidf[corpus]

## LSI

In [31]:
lsi_model = models.LsiModel.load('model/lsi_model.lsi')
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lsi = lsi_model[vec_bow] # 用前面建好的 lsi 模型去計算這一篇歌詞 (input: 斷詞後的詞包、output: 20個主題成分)

In [34]:
index = similarities.MatrixSimilarity(lsi_model[corpus])  # 建立索引
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key = lambda item: -item[1])

In [39]:
names, lyrics, top10 = [], [], []
for n, line, ten in zip(list(df['Name']), list(df['Lyric']), list(df['top_10'])):
    names.append(n)
    lyrics.append(line)
    top10.append(ten)

for lyric in sims[:5]:
    print("\n相似歌詞: ",  names[lyric[0]])
    print("TF-IDF top 10: ", top10[lyric[0]])
    print(lyrics[lyric[0]])
    print("相似度：",  lyric[1])


相似歌詞:   Baby Song
TF-IDF top 10:  ['進風裡', '我們', '小小的', '像顆', '裡面', '一個', '無窮', '夢裡', '那麼', '通透']
你的眼睛 像顆水晶通透裡面有一個無窮無盡的宇宙小小的你 在你小小的夢裡把我所有大大的事情都吹進風裡我為我將 對你撒的謊先跟你道歉當你發現黑白不是那麼的分明世界 不是那麼的公平別太失望 我講的是個夢想不用太聽我們的話不要讓 任何人告訴你你該怎樣對待世界或它該怎對你  要跟現在一樣隨心讓你的眼睛和心依然純淨可惜 世界不及你好原諒我們 我們都還在找而時間它只負責流動不負責育你成長不過你只需要 傾聽傾聽你的心你的眼睛 像顆水晶通透裡面有一個無窮無盡的宇宙小小的你 在你小小的夢裡把我所有大大的事情 都吹進風裡都吹進風裡
相似度： 0.999207

相似歌詞:   心的距離
TF-IDF top 10:  ['我忘記', '時間', '沒有', '機會', '哪個', '邊緣', '空間裡', '不斷', '怎麼', '直線']
看前面　我忘記了是哪個夏天你輕靠著我　飄散而過的落葉為了誓言　讓時間延伸就像永遠遲鈍如我　也感覺到的邊緣在思念的空間裡不斷徘徊那距離卻越明顯持續的提醒我　現實的界限 又一遍　我忘記了是哪些時間　你言詞閃爍　原因當然不明顯　試著看見　讓時間倒轉回到從前　認真如我　有抓不到的邊緣 　在想像的空間裡不斷徘徊　那畫面永遠明確　就算是閉上眼　也無法否決 我怎麼會讓自己捨身不斷涉險　你怎麼會對我的心不斷的拒絕  　愛失去你的包圍　每次退後又錯過你的世界一點 　我沒有辦法清醒應付新的對決　你卻輕易讓我的心委屈到極限　愛有了你　卻失去了我的一切　衡量你的心直線到我之間　沒有跨越的機會 沒有跨越的機會衡量你的心直線到我之間沒有跨越的機會看前面　我忘記了是哪個夏天
相似度： 0.9989911

相似歌詞:   掉了
TF-IDF top 10:  ['沒有', '笑靨', '眼淚', '出現', '瞬間', '顏色結', '回憶', '敵人', '節掉', '對白']
心疼的玫瑰 半夜還開著找不到匆匆掉落的花蕊回到現場卻 已來不及等待任何回音都不可得微弱的風箏 冬天裡飄著回不去手中纏線的那個沒有藍天 又何必去飛怎麼適合黑色笑靨掉了雪白眼淚掉了該出現的所有表情瞬間掉了瞳孔沒有顏

## LDA

In [40]:
lda_model = models.LsiModel.load('model/lda_model.lda')
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lsi = lda_model[vec_bow] # 用前面建好的 lsi 模型去計算這一篇歌詞 (input: 斷詞後的詞包、output: 20個主題成分)

In [41]:
index = similarities.MatrixSimilarity(lda_model[corpus])  # 建立索引
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key = lambda item: -item[1])

In [42]:
names, lyrics, top10 = [], [], []
for n, line, ten in zip(list(df['Name']), list(df['Lyric']), list(df['top_10'])):
    names.append(n)
    lyrics.append(line)
    top10.append(ten)

for lyric in sims[:5]:
    print("\n相似歌詞: ",  names[lyric[0]])
    print("TF-IDF top 10: ", top10[lyric[0]])
    print(lyrics[lyric[0]])
    print("相似度：",  lyric[1])


相似歌詞:   Show The World
TF-IDF top 10:  ['Go', 'world', 'Show', 'Goal', 'Let', '機會', '越過', '攔截', '歡呼將', '殲滅']
Go Go Go 機會就近在眼前越過那些狂野攔截Go Go Go 歡呼將殲滅疲倦瞬間移動如同閃電Goal Let's Go! 情勢多惡劣危險兄弟就是最強防線 Wo~~Goal Let's Go! 對平凡極度厭倦跳入一個傳奇裡面  昔日的追風少年 不乾的熱血脫下白襯衫 心才被看見熱愛不止于球場 在生活蔓延越激烈越熱切 敢上就敢對決Go Go GoShow the world 我的火光 我的信仰Show the world 持續補強 急速成長Show the world 我的渴望 我的戰場Show the world 誰在雲上 誰會稱王
相似度： 0.9941994

相似歌詞:   心腹
TF-IDF top 10:  ['Ba', '記住', 'BaDa', '通向', '一個', '距離', 'lovin', '心腹', '溶化', '假期']
口　通向腹　通向心　通向彼此憶記每幕都這麼美他　邀約她　邀約他　吃著早餐等你我在餐廳識你記住溶化的距離　記住相通的天地BaDa Ba Ba Ba每場痛快歷奇記住完美的假期　記住一起的滋味BaDa Ba Ba Ba耳目心腹手口鼻結伴分享色香味BaDa Ba Ba Ba Ba一班好友　叫做手足嗎　心腹嗎一餐歡宴　熟人要有吧　送暖遞茶一口小吃　會直通心嗎一番體貼　一點親切　未來也有吧　至老掉牙給我一個包　你一個包你跟我不結交　也可吃飽最好笑的笑話　卻不會包最好你跟我可以緊靠說著笑著吃包包你　逐條味蕾末梢馬上化做快樂氣泡　飛上月球做巢口　通向腹　通向心　通向彼此憶記以食譜寫週記記住溶化的距離　記住相通的天地BaDa Ba Ba Ba每場痛快歷奇記住完美的假期　記住一起的滋味BaDa Ba Ba Ba每啖都想分給你  BaDa Ba Ba Ba一班好友　叫做手足嗎　心腹嗎一餐歡宴　熟人要有吧　送暖遞茶一口小吃　會直通心嗎一番體貼　一點親切　未來也有吧　至老掉牙給我一個包　你一個包你跟我不結交　也可吃飽最好笑的笑話　卻不會包最好你跟我可以緊靠說著笑著吃包包你　逐條味蕾末梢馬上化做快樂氣泡　飛上月球做巢記住溶化的距離　記